## Mount Google Drive

In [ ]:
from google.colab import drive

# Make sure to unmount drive at mount point
drive.flush_and_unmount()
drive.mount('/content/drive')

# Research Start

In [1]:
from datetime import datetime
from scipy import stats  
import os
import pytz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np  

%matplotlib inline

### Creating IDX Equities Time Series

In [2]:
# idx_combined = pd.read_csv('/home/nikki/documents/amibroker/idx_exported_combined.csv', parse_dates=[1])

In [3]:
# Location of the IDX Stocks OHLC csv files
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv'
# csv_loc = '/Users/nikki/Documents/idx_exported_csv'
csv_loc = '/home/nikki/documents/amibroker/exported-csv/'

# Backup if the first one doesn't work
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv_gama'

tickers = os.listdir(csv_loc)

In [4]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
data_idx = {}
for ticker in tickers:
    data_idx[ticker[:-4]] = pd.read_csv(f'{csv_loc}/{ticker}',
                                        index_col='date',
                                        parse_dates=['date'])

In [5]:
data_idx['ASII'].tail()

,open,high,low,close,volume
date,,,,,
2020-09-07,5050.0,5075.0,4950.0,5025.0,37120700
2020-09-08,5025.0,5125.0,5025.0,5125.0,22790200
2020-09-09,5025.0,5025.0,4900.0,4900.0,62626200
2020-09-10,4560.0,4610.0,4560.0,4560.0,142910704
2020-09-11,4350.0,4700.0,4310.0,4630.0,76725696


### Stocks on the Move


In [6]:
def momentum_score(ts):
    """
    Input:  Price time series.
    Output: Annualized exponential regression slope, 
            multiplied by the R2
    """
    # Make a list of consecutive numbers
    x = np.arange(len(ts)) 
    # Get logs
    log_ts = np.log(ts) 
    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # Annualize percent
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    #Adjust for fitness
    score = annualized_slope * (r_value ** 2)
    return score

In [7]:
def volatility(ts, period=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(period).std().iloc[-1]

In [8]:
# How many (series) candles back for momentum calculation?
momentum_window = 96

# Create an empty DataFrame to store score
momentum_table = pd.DataFrame(columns=['ticker', 'score', 'vola', 'inv_vola'])

# Eliminated stocks list
eliminated_stocks = pd.DataFrame(columns=['ticker', 'score', 'vola', 'reason'])

# How many (series) candles+1 back for std dev calculation?
vola_window = 24

# How many (series) candles back for EWMA calculation?
ewma_period = 128

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, timeseries in data_idx.items():
    momentum_series = timeseries['close'].iloc[-momentum_window:]
    score = momentum_score(momentum_series)
    vola_series = timeseries['close']
    vola = volatility(vola_series, vola_window) * 16
    median_volume = timeseries['volume'].rolling(vola_window).median().iloc[-1]
    ewma = timeseries['close'].ewm(span=ewma_period).mean().iloc[-1]

    # Need the stocks to exist at least 3 years prior (756 trading days)
    if len(timeseries) < 756:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'umur belum 3 tahun'},
                                                     ignore_index=True)
        continue
    
    # If median volume falls below 100k in the stocks, drop it 
    if median_volume < 100000:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'avg volume di bawah 100k'},
                                                     ignore_index=True)
        continue
        
    # If it has been suspended (daily vol == 0) more than once, drop it
    if timeseries['volume'].iloc[-momentum_window:].tolist().count(0) > 1:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'pernah disuspend lebih dari 1x'},
                                                     ignore_index=True)
        continue
    
    momentum_table = momentum_table.append({'ticker': ticker,
                                            'score': score,
                                            'vola': vola,
                                            'median_vol': median_volume,
                                            'ewma': ewma},
                                           ignore_index=True)
    
    momentum_table['inv_vola'] = 1 / momentum_table['vola']

In [9]:
print(f'Ada {len(momentum_table)} saham lolos')
print(f'Ada {len(eliminated_stocks)} saham tereliminasi')

Ada 211 saham lolos
Ada 492 saham tereliminasi


In [10]:
momentum_table.sort_values('score', ascending=False)[:50].to_clipboard()

In [11]:
momentum_table.sort_values('score', ascending=False)[:20]

,ticker,score,vola,inv_vola,ewma,median_vol
60,PYFA,6143.004927,0.488737,2.046089,610.899936,1742000.0
12,WIIM,4131.396640,0.651467,1.534998,231.430917,36738700.0
184,AGRO,3973.188969,0.846056,1.181955,210.954304,71099902.0
40,INAF,2935.059357,0.624109,1.602284,1851.813539,4649150.0
121,CENT,2484.972986,0.650940,1.536239,96.099628,17894850.0
30,MARK,1951.295582,0.550642,1.816062,646.855387,5854900.0
169,BBHI,1544.426699,0.941086,1.062602,122.117340,144750.0
33,KAEF,1521.306644,0.636893,1.570123,1997.140863,18072500.0
87,KRAS,1331.743771,0.388212,2.575912,306.130276,39877400.0
139,DOID,1136.504910,0.712931,1.402659,219.360688,75969748.0


In [12]:
momentum_table.sort_values('score', ascending=False)[:20]

,ticker,score,vola,inv_vola,ewma,median_vol
60,PYFA,6143.004927,0.488737,2.046089,610.899936,1742000.0
12,WIIM,4131.396640,0.651467,1.534998,231.430917,36738700.0
184,AGRO,3973.188969,0.846056,1.181955,210.954304,71099902.0
40,INAF,2935.059357,0.624109,1.602284,1851.813539,4649150.0
121,CENT,2484.972986,0.650940,1.536239,96.099628,17894850.0
30,MARK,1951.295582,0.550642,1.816062,646.855387,5854900.0
169,BBHI,1544.426699,0.941086,1.062602,122.117340,144750.0
33,KAEF,1521.306644,0.636893,1.570123,1997.140863,18072500.0
87,KRAS,1331.743771,0.388212,2.575912,306.130276,39877400.0
139,DOID,1136.504910,0.712931,1.402659,219.360688,75969748.0


In [13]:
eliminated_stocks[eliminated_stocks['reason'] == 'umur belum 3 tahun'].sort_values('score', ascending=False)

,ticker,score,vola,reason
39,BRIS,6259.221389,1.030049,umur belum 3 tahun
226,DIVA,1676.213720,0.520243,umur belum 3 tahun
81,DMMX,1168.078239,0.513641,umur belum 3 tahun
145,MCAS,1054.824595,0.420651,umur belum 3 tahun
43,CBMF,1025.888629,0.696835,umur belum 3 tahun
...,...,...,...,...
337,INPS,-79.678575,1.169546,umur belum 3 tahun
0,ITIC,-91.986955,0.541924,umur belum 3 tahun
206,ARKA,-93.327321,0.882876,umur belum 3 tahun
383,DADA,-94.054739,0.530424,umur belum 3 tahun


In [14]:
eliminated_stocks[eliminated_stocks['reason'] == 'pernah disuspend lebih dari 1x'].sort_values('score', ascending=False)

,ticker,score,vola,reason
160,TIFA,1564.864686,0.337551,pernah disuspend lebih dari 1x
256,INCI,564.798381,0.478901,pernah disuspend lebih dari 1x
108,WINS,136.481318,0.168128,pernah disuspend lebih dari 1x
476,BIPP,89.717771,0.534408,pernah disuspend lebih dari 1x
38,RMBA,88.611302,1.683493,pernah disuspend lebih dari 1x
302,RUIS,69.980301,1.270678,pernah disuspend lebih dari 1x
127,RANC,57.318397,0.335778,pernah disuspend lebih dari 1x
272,BTON,41.610341,0.576428,pernah disuspend lebih dari 1x
59,LPPS,8.060772,0.590150,pernah disuspend lebih dari 1x
26,APIC,0.059742,0.402388,pernah disuspend lebih dari 1x


In [15]:
eliminated_stocks[eliminated_stocks['reason'] == 'avg volume di bawah 100k'].sort_values('score', ascending=False)

,ticker,score,vola,reason
329,ARTO,5561.937080,0.294194,avg volume di bawah 100k
76,ARGO,1026.216090,1.078083,avg volume di bawah 100k
352,PNSE,333.866671,0.063360,avg volume di bawah 100k
441,BCIC,219.618961,0.000000,avg volume di bawah 100k
364,BBMD,205.920303,0.575655,avg volume di bawah 100k
...,...,...,...,...
69,PGLI,-60.902342,1.499969,avg volume di bawah 100k
229,AMIN,-66.796392,0.845457,avg volume di bawah 100k
430,ABMM,-67.759026,0.278749,avg volume di bawah 100k
366,GLOB,-72.854758,0.844873,avg volume di bawah 100k
